# Multiple-criteria decision analysis (MCDA) with Evolutionary Methods
By Sina Azartash<br>
Johns Hopkins University: Whiting School of Eningeering <br>
Fall 2021 Reasoning Under Uncertaintity by Marc Johnson <br>

###Using Desdeo-emo
Credit to Industrial Optimization Group, University of Jyväskylä, Finland<br>
 http://www.mit.jyu.fi/optgroup/<br>
https://github.com/industrial-optimization-group


In [6]:
!pip install desdeo_emo   #requires the latest update of numpy

     |████████████████████████████████| 82 kB 582 kB/s 
     |████████████████████████████████| 2.0 MB 19.2 MB/s 
     |████████████████████████████████| 23.1 MB 1.7 MB/s 
     |████████████████████████████████| 323 kB 49.7 MB/s 
     |████████████████████████████████| 3.4 MB 52.8 MB/s 
     |████████████████████████████████| 25.3 MB 87 kB/s 
  Created wheel for diversipy: filename=diversipy-0.8-py3-none-any.whl size=27531 sha256=7ffaf4feb0743b5aaee8206c87e7ec5032d4f02742394dad603fbab517d005bb
  Stored in directory: /root/.cache/pip/wheels/43/af/e6/4964143acc3bb5893d9e188f091e31ab6f6c1a277739033a0e
  Created wheel for optproblems: filename=optproblems-1.3-py3-none-any.whl size=2040690 sha256=42fc8a6440a99a352394a908c06bcb2f7b2ad11079c703aeebd650699e193b90
  Stored in directory: /root/.cache/pip/wheels/3a/8d/1b/762402c6cbdfd012cf3622d55b6460440d90dafe59dad0c013
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18184 sha256=444fd6e93a776f43373074ed867613acf4c793e0b92b

In [7]:
pip install --upgrade numpy

     |████████████████████████████████| 15.7 MB 198 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires numpy~=1.19.2, but you have numpy 1.21.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [1]:


import plotly.graph_objects as go
import numpy as np
import pandas as pd

from desdeo_problem import variable_builder, ScalarObjective, MOProblem
from desdeo_problem.testproblems.TestProblems import test_problem_builder

from desdeo_emo.EAs.NSGAIII import NSGAIII
from desdeo_emo.EAs.RVEA import RVEA
from desdeo_emo.utilities.plotlyanimate import animate_init_, animate_next_



A single decision consists of multiple goals. To make the best decision, we must optimize the set of those goals. Specifically, we have to maximize goals we are trying to maximize reward and minimize cost and risk. Goals can be overlapping, conflicting and have different weights. This makes its analysis very complex. For this reason, we use evolutionary algorithms to find the optimum value of each of those goals. Goals can be mathematically defined as a scalr objective function. There is actually several optimum value sets for our set of objective sets presenting a list of maximally optimum solutions. This list of potential solutions is referred the Pareato Front and the algorithms below only minimize it.

### Define Objective Functions Here

Three example arbitrary functions have been defined below. Argument X is a 2D Numpy array

In [16]:
def goal_1(x):
    term1 = ((x[:,0] - 2) ** 2) / 2
    term2 = ((x[:,1] + 1) ** 2) / 13
    return term1 + term2 + 3

def goal_2(x):
    term1 = ((x[:, 0] + x[:, 1] - 3) ** 2) / 36
    term2 = ((-x[:, 0] + x[:, 1] + 2) ** 2) / 8
    return term1 + term2 - 17

def goal_3(x):
    term1 = ((x[:, 0] + (2 * x[:, 1]) - 1) ** 2) / 175
    term2 = ((-x[:, 0] + 2* x[:, 1]) ** 2) / 17
    return term1 + term2 - 13

def goal_4(x):
    term = x[:, 0]**2 + x[:, 1]**2
    return 0.5*term + np.sin(term)

def goal_5(x):
    term1 = ((3*x[:, 0] - 2*x[:,1] + 4)**2)/8
    term2 = ((x[:, 0] - x[:,1] + 1)**2)/27
    return term1 + term2 + 15

def goal_6(x):
    term = x[:, 0]**2 + x[:, 1]**2
    return (1/(term + 1)) - 1.1 * np.exp(-term)



### Define Search Space
Constraint: X >= -400 & x <=400<br>
Below, we set the origin as our starting point (x=0,y=0) <br>
Sometimes it is obvious that the value of a goal will be within a mathematical bounds. For example, we can be sure that the price of a car will never be negative, so we would eliminate negative numbers from our constraint.


In [17]:


list_vars = variable_builder(['x', 'y'],
                             initial_values = [0,0],
                             lower_bounds=[-400, -400],
                             upper_bounds=[400, 400])
list_vars



#### Investigate goals 1 , 2, and 3
Below, each objective function must be wrapped in Scalar Objective Class and then put in a parameter list

In [18]:


f1 = ScalarObjective(name='goal_1', evaluator=f_1)
f2 = ScalarObjective(name='goal_2', evaluator=f_2)
f3 = ScalarObjective(name='goal_3', evaluator=f_3)
list_objs = [f1, f2, f3]
problem = MOProblem(variables=list_vars, objectives=list_objs)


### Specify Parameters For Evolutionary Methods
@arg problem = the object containing of our objective functions<br>
@arg n_iterations = the number of generations to consider. The higher this number, the more accurate, but more running time required<br>
@arg population_size = the maximum number of individuals in a generation (the maximum number of solutions to consider at a time)<br>
@arg selection_type = uncertaintity estimate.

In [19]:
evolver = NSGAIII(problem,
                  n_iterations=10,
                  n_gen_per_iter=100,
                  population_size=100)

Runs the evolutionary aglrotihm

In [20]:


while evolver.continue_evolution():
    evolver.iterate()



Display the results on a 2D graph

In [21]:


individuals, solutions = evolver.end()

fig1 = go.Figure(
    data=go.Scatter(
        x=individuals[:,0], 
        y=individuals[:,1], 
        mode="markers"))
fig1



In [22]:
fig2 = go.Figure(data=go.Scatter3d(x=solutions[:,0],
                                   y=solutions[:,1],
                                   z=solutions[:,2],
                                   mode="markers",
                                   marker_size=5))
fig2

In [14]:


pd.DataFrame(solutions).to_csv("MOP7_true_front.csv")



#### Investigate goals #4,5 and 6

In [25]:
list_vars = variable_builder(['x', 'y'],
                             initial_values = [0,0],
                             lower_bounds=[-30, -30],
                             upper_bounds=[30, 30])
f1 = ScalarObjective(name='goal_4', evaluator=goal_4)
f2 = ScalarObjective(name='goal_5', evaluator=goal_5)
f3 = ScalarObjective(name='goal_6', evaluator=goal_6)
problem = MOProblem(variables=list_vars, objectives=[f1, f2, f3])

evolver = NSGAIII(problem)


Below we plot the fitness of each solution attempting to maximize the correct configuration of parameter values. Download the html file and then veiw using a internet browser. We can see that the fitness values start out that our approximiation of the pareto front starts out scattered and crude. After 10 iterations, it becomes much more refined.

In [26]:
individual, solutions = evolver.end()
figure = animate_init_(solutions, filename="MOP5.html")

Plot saved as:  MOP5.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.


In [27]:
while evolver.continue_evolution():
    print(f"Running iteration {evolver._iteration_counter+1}")
    evolver.iterate()
    non_dominated = evolver.population.non_dominated_fitness()
    figure = animate_next_(
        evolver.population.objectives[non_dominated],
        figure,
        filename="MOP5.html",
        generation=evolver._iteration_counter,
    )

Running iteration 1
Running iteration 2
Running iteration 3
Running iteration 4
Running iteration 5
Running iteration 6
Running iteration 7
Running iteration 8
Running iteration 9
Running iteration 10


### More Examples
These examples were taken from GitHub. I have included them below because I have not figured yet how to adapt and modify such as above. Below, is an example that estimated the pareto front for only 1 iteration.

In [37]:


problem = test_problem_builder(name="DTLZ1", n_of_variables=30, n_of_objectives=3)

evolver = RVEA(problem, interact=True, n_iterations=5, n_gen_per_iter=400)
figure = animate_init_(
    evolver.population.objectives[evolver.population.non_dominated_fitness()],
    filename="dtlz1.html")

Plot saved as:  dtlz1.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.


In [38]:
pref, plot = evolver.start()
response = evolver.population.ideal_fitness_val + [0.5,0.7,0.1]
pref[2].response = pd.DataFrame([response], columns=pref[2].content['dimensions_data'].columns)

In [39]:
pref, plot = evolver.iterate(pref[2])
figure = animate_next_(
    plot.content['data'].values,
    figure,
    filename="dtlz1.html",
    generation=evolver._iteration_counter,
)

message = (f"Current generation number:{evolver._current_gen_count}. "
           f"Is looping back recommended: {'Yes' if evolver.continue_evolution() else 'No'}")
print(message)

Current generation number:400. Is looping back recommended: Yes


#To Do:
Further description is needed under graphs
A string method should be creaated to explain context of a pareto optimal solution
A method should be created to slide between different pareto optimal solutions